In [1]:
# baseline cnn model for fashion mnist

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

#import stuff for the project

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix

#sklearn stuff importing

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#for data cleaning- organizing
import glob 

Using TensorFlow backend.
C:\Users\97252\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\97252\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\97252\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\97252\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
a1=pd.read_csv(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\Unity Data\Training\Evyatar Cohen\Evyatar636771052727603804Spontan.csv')
a1['State']= '1'
a1.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,State
0,335.9984,128556,right,2,128.63570,224.0294,-12.29588,-18.26243,-140.54450,-117.57790,...,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.0,1.250576,0.0,1
1,335.9984,128556,left,2,-93.24845,143.9423,-37.81597,-109.09820,240.38980,-67.62959,...,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0,1
2,336.0150,128558,right,2,129.15550,221.9923,-15.00181,47.36464,-64.25022,-168.85850,...,177.7894,171.1952,-14.677850,276.0645,-33.15913,97.88171,0.0,1.069930,0.0,1
3,336.0150,128558,left,2,-94.86554,148.5542,-39.00158,-80.99012,278.60220,-66.79356,...,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0,1
4,336.0315,128560,right,2,129.87070,220.7944,-18.38681,35.05422,-84.74242,-205.72030,...,178.3730,169.9159,-15.246240,276.1767,-30.72506,104.13920,0.0,0.916969,0.0,1


In [ ]:
#Spontan- as 1. Sync- as 2. Alone-as 3.

#PROCESSING THE TRAINING DATA 

i=1
dfs = pd.DataFrame()
dfs = dfs.fillna(0)

for path in glob.glob(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\Unity Data\Training\*'): 
    paths= path+"\*"
    for pathy in glob.glob(paths): 
        name=str(i)
        name=pd.read_csv(pathy)
        if "Spontan" in pathy:
            name['State']= '1'
        if "Sync" in pathy:
           name['State']= '2'
        if "Alone" in pathy:
           name['State']= '3'
        i += 1
        dfs = pd.concat([dfs, name])
        
#dfs.head() 
dfs.to_csv(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\dfs.csv', index = False) #checking if this is actually the dataset we wish to gey

dfs.columns = dfs.columns.str.strip() #dealing with error, caused by leading whistespaces in column names in the csv file

gp=dfs.groupby('Hand Type')

In [ ]:
#Right
righty= gp.get_group('right')
righty.columns = [str(col) + '_Right' for col in righty.columns]
righty

#Left
lefty= gp.get_group('left')
lefty.columns = [str(col) + '_Left' for col in lefty.columns]
lefty

#reset_indexes
righty=righty.reset_index(drop=True)
lefty=lefty.reset_index(drop=True)

#concat them
done=pd.concat([righty, lefty], axis=1)
done.to_csv(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\training set.csv', index = False) #checking if this is actually the dataset we wish to gey



In [ ]:
#editing the df
#removing unessucery columns, which will ruin the prediction process

done=done.drop('Frame ID_Right', axis=1)
done=done.drop(labels='Frame ID_Left', axis=1)
done=done.drop(labels='Hand Type_Right', axis=1) 
done=done.drop(labels='Hand Type_Left', axis=1)
done=done.drop(labels='# hands_Left', axis=1) 
done=done.drop(labels='# hands_Right', axis=1) 
done=done.drop(labels='Time_Left', axis=1) 
done=done.drop(labels='Time_Right', axis=1) 
done=done.drop(labels='State_Right', axis=1) 

done

In [ ]:
#fill NaNs with 0 so we can build a model

done=done.fillna(0)

#change the df from str to int to apply numeric calculations needed for the model

done=done.apply(pd.to_numeric, errors='ignore')

#done.columns = done.columns.str.strip().str.replace(' ', '').str.replace('(', '').str.replace(')', '')

done.info()

In [ ]:
#Graph: Training data's labels' distribution!

import seaborn as sns

sns.set(font_scale=1.3)  # crazy big

plt.figure(figsize=(10, 10))

axes = sns.countplot(y=done.State_Left)

csfont = {'fontname':'Comic Sans MS'}

axes.set_title("Training data's labels' distribution!",**csfont,fontsize=25)
axes.set_ylabel('Spontan- 1. Sync- 2. Alone- 3.',fontsize=20,**csfont)
axes.set_xlabel(' Number of occurances ',fontsize=20,**csfont)

fig1 = plt.gcf()
plt.show()
plt.draw()

#Seems like they're equally divided there!

In [ ]:
#tTEST dataset preparing

#gathering the dataset  from the Validation folder

#Spontan- as 1. Sync- as 2. Alone-as 3.

import pandas as pd
import glob #for collecting the datasets

i=1
dfsa = pd.DataFrame()
dfsa = dfsa.fillna(0)

for path in glob.glob(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\Unity Data\Validation\*'): 
    paths= path+"\*"
    for pathy in glob.glob(paths): 
        name=str(i)
        name=pd.read_csv(pathy)
        if "Spontan" in pathy:
            name['State']= '1'
        if "Sync" in pathy:
           name['State']= '2'
        if "Alone" in pathy:
           name['State']= '3'
        i += 1
        dfsa = pd.concat([dfsa, name])
        
#dfs.head() 
dfsa.to_csv(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\dfs.csv', index = False) #checking if this is actually the dataset we wish to gey

dfsa.columns = dfsa.columns.str.strip() #dealing with error, caused by leading whistespaces in column names in the csv file


gpa=dfsa.groupby('Hand Type')


#Hand Type: Right
rightya= gpa.get_group('right')
rightya.columns = [str(col) + '_Right' for col in rightya.columns]
rightya

#Hand Type: Left
leftya= gpa.get_group('left')
leftya.columns = [str(col) + '_Left' for col in leftya.columns]
leftya

#reset_indexes
rightya=rightya.reset_index(drop=True)
leftya=leftya.reset_index(drop=True)

#concat them
testy=pd.concat([rightya, leftya], axis=1)
testy.to_csv(r'C:\Users\97252\OneDrive\שולחן העבודה\הדנת- עבודת גמר\test set.csv', index = False) #checking if this is actually the dataset we wish to gey

In [ ]:
#editing the df
#removing unessucery columns, which will ruin the prediction process

testy=testy.drop('Frame ID_Right', axis=1)
testy=testy.drop(labels='Frame ID_Left', axis=1)
testy=testy.drop(labels='Hand Type_Right', axis=1) 
testy=testy.drop(labels='Hand Type_Left', axis=1)
testy=testy.drop(labels='# hands_Left', axis=1) 
testy=testy.drop(labels='# hands_Right', axis=1) 
testy=testy.drop(labels='Time_Left', axis=1) 
testy=testy.drop(labels='Time_Right', axis=1) 
testy=testy.drop(labels='State_Right', axis=1) 

testy

In [ ]:
#Graph: Testing data's labels' distribution!

import seaborn as sns

sns.set(font_scale=1.3)  # crazy big

plt.figure(figsize=(10, 10))

axes = sns.countplot(y=testy.State_Left)

csfont = {'fontname':'Comic Sans MS'}

axes.set_title("Testing data's labels' distribution!",**csfont,fontsize=25)
axes.set_ylabel('Spontan- 1. Sync- 2. Alone- 3.',fontsize=20,**csfont)
axes.set_xlabel(' Number of occurances ',fontsize=20,**csfont)

fig1 = plt.gcf()
plt.show()
plt.draw()

#Seems like they're equally divided there too!

In [ ]:
#Graph: Showing correlation between the columns in the training data!

#Pandas dataframe.corr() is used to find the pairwise correlation of all columns in the dataframe.
#Any na values are automatically excluded. 
#For any non-numeric data type columns in the dataframe it is ignored
#Note: The correlation of a variable with itself is 1.

#Using corr() function to find the correlation among
#the columns in the dataframe using ‘kendall’ method:

fig = plt.figure(figsize=(30,30))
axes=sns.heatmap(done.corr(method ='kendall'),annot=True,square=True,cmap="gist_rainbow_r", linewidths=.5)
axes.set_title("Correlation between The Coloumns: Training data!",**csfont,fontsize=50)

plt.show()

In [ ]:
#Let's fill NaNs with 0 so we can build a model

testy=testy.fillna(0)

#change the df from str to int to apply numeric calculations needed for the model

testy=testy.apply(pd.to_numeric, errors='ignore')

testy.info()

In [ ]:
#the cnn stuff we can put back if needed 

In [ ]:
#Reading the Data and Performing Basic Data Checks

df=done #this is the training set of this project
dfa=testy #this is the testing set of this project

df.head()

In [ ]:
dfa.head()

In [ ]:
#Creating Arrays for the Features and the Response Variable.

target_column = ['State_Left'] 
predictors = list(set(list(df.columns))-set(target_column))
#print(predictors)
df[predictors] = df[predictors]/df[predictors].max()
#print(df.describe())

In [ ]:
#Creating the Training and Test Datasets

#train

X_train = df[predictors].values
y_train = df[target_column].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

#test
Xa_test = dfa[predictors].values
ya_test = dfa[target_column].values

#Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, test_size=0.30, random_state=40)

print('X_train.shape is ',X_train.shape); print('Xa_test.shape is ',Xa_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

y_train=y_train.reshape(-1, 1) #make it 1d array for the function
# Split the data (X, Y)
from sklearn import metrics
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train = df[predictors].values
y_train = df[target_column].values

Xa_test = dfa[predictors].values #it doesn't get one-hot encoded, so we take the real values as they were at the beginning
ya_test = dfa[target_column].values

#Creating and training the model
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(Xa_test)
print("\n Logistic Regression- Score is %.3f" % logmodel.score(Xa_test, ya_test))

#Evaluating the model
cfm = metrics.confusion_matrix(ya_test, predictions)
print(cfm)
print('')
#    sns.heatmap(cfm, annot=True)
print(classification_report(ya_test, predictions))

In [ ]:
# PREDICTION BY ALGORITHMS:
# KNeighbors
#DecisionTree
#RandomForest
#Adaboost
#Gradient Boosting

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

classifiers = [
    KNeighborsClassifier(10),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
for classifier in classifiers:
    model = classifier.fit(X_train, y_train)
    print("\n The Classifier is \n " +str(classifier))
    print("\n The Model's Score is %.3f" % model.score(Xa_test, ya_test))
    y_pred_best = model.predict(Xa_test)
    
    cm=confusion_matrix(ya_test, y_pred_best, labels=None, sample_weight=None)
    print(cm)
    print('')
    
    #cross validation
    print(classification_report(ya_test,y_pred_best))
    if (str(classifier) == 'KNeighborsClassifier(n_neighbors=10)'):  #CROSS VALIDATION! activate only KNN
        score= cross_val_score(classifier, X_train, y_train, cv=4)
        print("the Cross Validation score of this model is ", score)
        
    #sns.heatmap(pd.DataFrame(classification_report).iloc[:-1, :].T, annot=True)

In [ ]:
#Nuerual Network classification model 
#Goal: predicting whether the person is talking in a sync, alone or spontan mode

#Loading the Required Libraries and Modules

# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

In [ ]:
#PREAPRING THE DATA FOR THE CNN MODEL!

#Since our target variable represents a binary category which has been coded as numbers 0 and 1,
#we will have to encode it. We can easily achieve that using the "to_categorical" function
#from the Keras utilities package.

#The two lines of code below accomplishes that in both training and test datasets.
# one hot encoded outputs:

import numpy

y_train = to_categorical(y_train)
ya_test = to_categorical(ya_test)

In [ ]:
#Define, Compile, and Fit the Keras Classification Model

model = Sequential()
model.add(Dense(500, activation='relu', input_dim=36))  #the input layer which specifies the activation function and the number of input dimensions, which in our case is 36 predictors.
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(4, activation='softmax')) #The fifth line of code creates the output layer with four nodes because there are four output classes

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# normalize to range 0-1
X_train = X_train / 255.0
Xa_test = Xa_test / 255.0


# build the model
model.fit(X_train, y_train, epochs=3)

#Predict on the Test Data and Compute Evaluation Metrics;

#pred_train= model.predict(X_train)
#scores2 = model.evaluate(X_train, y_train, verbose=0)
#print('Accuracy on train data: {}% \n Error on train data: {}'.format(scores2[1], 1 - scores2[1]))    

print('')

pred_test= model.predict(Xa_test)
scores = model.evaluate(Xa_test, ya_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1]))   



In [ ]:
#get precision, recall and F-SCORE

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(X_train, y_train, validation_split=0.3, epochs=10, verbose=0)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(Xa_test, ya_test, verbose=0)

#cfm = metrics.confusion_matrix(ya_test, predictions)
#sns.heatmap(cfm, annot=True)

print("The model's F-SCORE on test set is ",f1_score)
print('')
print("The model's precision on test set is ",precision)
print('')
print("The model's recall on test set is ",recall)